In [1]:
from datasets import Dataset, load_dataset, concatenate_datasets

## Load Dataset

In [2]:
news_tr=load_dataset("habanoz/news-tr-90k")

In [3]:
news_tr

DatasetDict({
    train: Dataset({
        features: ['Id', 'Tag', 'Labels', 'Title', 'Summary', 'Text', '__index_level_0__'],
        num_rows: 90787
    })
})

In [4]:
news_tr_cultur = news_tr.filter(lambda x: 'kultur-sanat' == x['Tag'])

Filter:   0%|          | 0/90787 [00:00<?, ? examples/s]

In [6]:
news_tr_cultur

DatasetDict({
    train: Dataset({
        features: ['Id', 'Tag', 'Labels', 'Title', 'Summary', 'Text', '__index_level_0__'],
        num_rows: 2502
    })
})

In [8]:
news_tr_cultur['train'][0]

{'Id': 508735,
 'Tag': 'kultur-sanat',
 'Labels': 'Nevşehir,Tarihi kilise',
 'Title': "'Hristiyan alemini heyecanlandıracak keşif'",
 'Summary': 'Nevşehir Belediye Başkanı Ünver, bulunan yer altı şehriyle ilgili, "Dünya Ortodoksluğu\'nun tarihi akışına yön verecek bulgularla karşı karşıyayız" dedi.',
 'Text': '**Arkeolojik sit alanı** ilan edilen, kazı çalışmalarının sürdüğü alanda bulunan tarihi kiliseyle ilgili gazetecilere açıklamada bulunan Nevşehir Belediye Başkanı Hasan Ünver, "**Dünya Ortodoksluğu** \'nun tarihi akışına yön verecek bulgularla karşı karşıyayız. Tamamıyla yerin altında kalıp içi toprak dolduğu için **fresk** lerde (Yaş duvar sıvası üzerine kireç suyunda eritilmiş madeni boyalarla resim yapma yöntemi) pek bozulma olmamış. İlk verilere göre milattan sonra 5\'inci yüzyıla ait görünüyor. Hristiyan dünyasında **heyecan** uyandıracak bulgulara ulaştık" diye konuştu.\n\nÜnver, **kilise**nin içinde **İsa Peygamber** \'in gökyüzüne yükseldiğini ve kötü ruhun öldürülmesinin

In [9]:
euronews_tr=load_dataset("habanoz/euronews-tr-v1.0")

Generating train split:   0%|          | 0/164620 [00:00<?, ? examples/s]

In [10]:
euronews_tr_cultur = euronews_tr.filter(lambda x: 'kultur' == x['Tag'])

Filter:   0%|          | 0/164620 [00:00<?, ? examples/s]

In [12]:
euronews_tr_cultur

DatasetDict({
    train: Dataset({
        features: ['Id', 'Tag', 'Labels', 'Title', 'Summary', 'Text', '__index_level_0__'],
        num_rows: 2713
    })
})

## Combine Datasets

In [15]:
culter_tr = concatenated_datasets = concatenate_datasets([news_tr_cultur['train'].remove_columns(['Id', 'Tag', '__index_level_0__']), 
                                                          euronews_tr_cultur['train'].remove_columns(['Id', 'Tag', '__index_level_0__'])]
                                                         )

In [16]:
culter_tr

Dataset({
    features: ['Labels', 'Title', 'Summary', 'Text'],
    num_rows: 5215
})

## Generate Article Dataset

In [141]:
prompt = "Generate a news article in Turkish about the following title:\n\nTitle:{}\nLabels:{}"

In [146]:
culter_article_ds = culter_tr.shuffle(123).select(range(4000, len(culter_tr))).map(lambda x: {'conversation': [{'role':'user', 'content': prompt.format(x['Title'], x['Labels'])}, {'role':'assistant', 'content': x['Text']}]}, remove_columns=culter_tr.column_names)

Map:   0%|          | 0/1215 [00:00<?, ? examples/s]

In [148]:
culter_article_ds

Dataset({
    features: ['conversation'],
    num_rows: 1215
})

In [149]:
culter_article_ds['conversation'][0]

[{'content': 'Generate a news article in Turkish about the following title:\n\nTitle:Abdalların efsane müzisyeni: Neşet Ertaş\nLabels:abdal,Abdallar Köyü,Anadolu efsanesi,anma,Bozkırın tezenesi,Kırşehir,Neşet Ertaş,Vefat',
  'role': 'user'},
 {'content': '#### "Bozkırın Tezenesi", "Anadolu Efsanesi" ve "Abdal Müzisyen" lakaplarıyla da müzik dünyasında tanınan halk ozanı, abdallık geleneğinin son temsilcilerinden Neşet Ertaş, yaşamı boyunca ürettiği unutulmaz eserleri Türk müziğine miras bıraktı.\n###### İstanbul\nTürküleri kendine has üslubuyla icra eden **Neşet Ertaş,** saz üstadı Muharrem Ertaş ile Döne Hanım\'ın oğlu olarak Kırşehir\'in Çiçekdağı ilçesine bağlı Abdallar (Kırtıllar) Köyü\'nde 1938\'de dünyaya geldi.\n\nBabası sayesinde müziğe ilgi duyan sanatçı, yaptığı bir açıklamada, o günleri, "Düğünlerde kaşık tutamayan küçüklere zil verilirdi. Babam bana da zil vermişti. Ben aşağı yukarı 4-5 sene babamın yanında zil çaldım. Zille başladım, kaşık da tuttum sonunda. Eskiden saz ne

In [150]:
sys_prompt = "Your are a helpful assistant. Generate a news article in Turkish about the given title and labels"

In [144]:
culter_article_chat_ds = culter_tr.shuffle(547).take(4000).map(
    lambda x: {
        'conversation':[ [{'role': 'system', 'content': sys_prompt}] + 
        [item for i in range(len(x['Title'])) for item in [
            {'role': 'user', 'content': f"Başlık: {x['Title'][i]}\nEtiketler: {x['Labels'][i]}"}, 
            {'role': 'assistant', 'content': x['Text'][i]}
        ]]
        ]
    },
    remove_columns=culter_tr.column_names, 
    batched=True, 
    batch_size=2
)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

In [145]:
culter_article_chat_ds

Dataset({
    features: ['conversation'],
    num_rows: 2000
})

In [129]:
culter_article_chat_ds[0]

{'conversation': [{'content': 'Your are a helpful assistant. Generate a news article in Turkish about the given title and labels',
   'role': 'system'},
  {'content': 'Başlık: Bu hafta 7 film vizyona girecek\nEtiketler: sinema',
   'role': 'user'},
  {'content': 'Bu hafta 2\'si yerli 7 film vizyona girecek.\n\nÖdüllü yönetmen Spike Jonze\'un, Altın Küre\'den "En iyi Senaryo", Amerikan Film Enstitüsü\'nden (AFI) "En İyi Film" ödülleri kazanan uzun metrajlı filmi "Aşk", izleyici ile buluşuyor.\n\nYalnızlık ve yaratıcılık sıkıntısı çeken bir yazarın dram ve komediyle yoğrulan öyküsünü beyaz perdeye taşıyan filmin başrolünde Joaquin Phoenix yer alırken, Scarlett Johansson da gizemli bilgisayar uygulamasına sesiyle hayat veriyor.\n\n**"Vampir Akademisi"**\n\nVampir filmi hayranları tarafından merakla beklenen "Vampir Akademisi" filmi, 35 ülkede çok satanlar listesinde uzun süre yerini koruyan altılı seri "Vampir Akademisi"nin sinemaya uyarlanan ilk bölümü. Tüm dünyada 14 Şubat\'ta vizyona g

## Generate Summary Dataset

In [151]:
prompt = "Generate a summary in Turkish language for a given article in Turkish.\n\nArticle:\n"

In [152]:
import re
def generate_article(example):
    text = example['Text'].replace(example['Summary'],'')
    text = re.sub(r'^[#\s]+', '', text) # remove leading # and spaces
    return {'conversation': [{'role':'user', 'content': prompt+text}, {'role':'assistant', 'content': example['Summary']}]}

In [156]:
news_summary_ds = culter_tr.map(generate_article, remove_columns=culter_tr.column_names)

Map:   0%|          | 0/5215 [00:00<?, ? examples/s]

In [157]:
news_summary_ds

Dataset({
    features: ['conversation'],
    num_rows: 5215
})

## Generate Final Dataset

In [158]:
tur_news_ds = concatenate_datasets([culter_article_ds, culter_article_chat_ds, news_summary_ds])

In [159]:
tur_news_ds

Dataset({
    features: ['conversation'],
    num_rows: 8430
})

In [ ]:
tur_news_ds.push_to_hub("habanoz/cultur_articles_tr", private=False, token="xxx")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/9 [00:00<?, ?ba/s]

Uploading files as a binary IO buffer is not supported by Xet Storage. Falling back to HTTP upload.


CommitInfo(commit_url='https://huggingface.co/datasets/habanoz/cultur_articles_tr/commit/20fa21894e96204cfeff9f29008a404cbdeeb747', commit_message='Upload dataset', commit_description='', oid='20fa21894e96204cfeff9f29008a404cbdeeb747', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/habanoz/cultur_articles_tr', endpoint='https://huggingface.co', repo_type='dataset', repo_id='habanoz/cultur_articles_tr'), pr_revision=None, pr_num=None)